In [1]:
import sys
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
from datetime import datetime

# 統計與科學計算
try:
    from scipy.stats import mannwhitneyu
    SCIPY_OK = True
except:
    SCIPY_OK = False

# 機器學習核心
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, RFECV

from sklearn.model_selection import (
    StratifiedKFold, train_test_split, cross_validate,
    GridSearchCV, RandomizedSearchCV, learning_curve
)

# 模型
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    VotingClassifier, StackingClassifier, AdaBoostClassifier
)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# 處理不平衡
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek

# 評估指標
from sklearn.metrics import (
    roc_auc_score, average_precision_score, f1_score,
    precision_recall_curve, roc_curve, confusion_matrix,
    classification_report, make_scorer
)

# 視覺化設定
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# ==================== 全域設定 ====================
print("Python version:", sys.version)
print("=" * 100)
print("完整改善版機器學習分析 - 啟動")
print("=" * 100)

Python version: 3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:42:04) [MSC v.1943 64 bit (AMD64)]
完整改善版機器學習分析 - 啟動


In [2]:
# 路徑設定
BASE_DIR = Path(r"D:\FLY114")
XLSX_PATH = BASE_DIR / "Diagnosis and autonomic marker data for VNS research_20250813.xlsx"

# 輸出資料夾
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT_DIR = BASE_DIR / f"Analysis_Results_{timestamp}"
PLOTS_DIR = OUT_DIR / "plots"
MODELS_DIR = OUT_DIR / "trained_models"
REPORTS_DIR = OUT_DIR / "reports"

for d in [OUT_DIR, PLOTS_DIR, MODELS_DIR, REPORTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

SHEET_NAME = "Sheet1"

# 欄位定義
BASIC_COLS = ["Age", "Sex", "BMI"]
LABEL_COLS = ["SSD", "MDD", "Panic", "GAD"]
CONTROL_COLS = ["DM", "TCA", "MARTA"]
HRV_COLS = ["SDNN", "LF", "HF", "LFHF", "SC", "FT", "RSA"]
ALL_FEATURES = BASIC_COLS + CONTROL_COLS + HRV_COLS

In [3]:
# ==================== 1. 資料載入與清理 ====================
print("\n[步驟 1] 資料載入與清理")
print("-" * 100)

df = pd.read_excel(XLSX_PATH, sheet_name=SHEET_NAME)
df.columns = [c.strip() for c in df.columns]
print(f"✓ 原始資料形狀: {df.shape}")

# 型別轉換
for c in [*ALL_FEATURES, *LABEL_COLS]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 處理 Sex
if "Sex" in df.columns and df["Sex"].dtype == object:
    norm = df["Sex"].astype(str).str.lower().str[0].map({"m": 1, "f": 0})
    if norm.notna().mean() > 0.6:
        df["Sex"] = norm

# LFHF >= 0
if "LFHF" in df.columns:
    df.loc[df["LFHF"] < 0, "LFHF"] = np.nan

# 離群值處理 (使用 IQR 方法)
def remove_outliers_iqr(df, cols, factor=3.0):
    df_clean = df.copy()
    outlier_report = []
    
    for col in cols:
        if col in df_clean.columns:
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - factor * IQR
            upper = Q3 + factor * IQR
            outliers = ((df_clean[col] < lower) | (df_clean[col] > upper))
            n_outliers = outliers.sum()
            
            if n_outliers > 0:
                outlier_report.append({
                    'feature': col,
                    'n_outliers': n_outliers,
                    'pct': f"{100*n_outliers/len(df):.2f}%"
                })
                df_clean.loc[outliers, col] = np.nan
    
    if outlier_report:
        print("\n離群值移除報告:")
        print(pd.DataFrame(outlier_report).to_string(index=False))
    
    return df_clean

numerical_cols = [c for c in HRV_COLS + ["Age", "BMI"] if c in df.columns]
df = remove_outliers_iqr(df, numerical_cols, factor=3.0)

print(f"✓ 清理後資料形狀: {df.shape}")


[步驟 1] 資料載入與清理
----------------------------------------------------------------------------------------------------
✓ 原始資料形狀: (502, 17)

離群值移除報告:
feature  n_outliers    pct
   SDNN          24  4.78%
     LF          53 10.56%
     HF          51 10.16%
   LFHF          28  5.58%
     SC          19  3.78%
     FT           1  0.20%
    RSA          25  4.98%
    BMI           2  0.40%
✓ 清理後資料形狀: (502, 17)


In [4]:
# ==================== 2. 特徵工程 ====================
print("\n[步驟 2] 特徵工程")
print("-" * 100)

engineered_features = []

# 交互特徵
if all(c in df.columns for c in ["LF", "HF"]):
    df["LF_HF_ratio"] = df["LF"] / (df["HF"] + 1e-9)
    df["LF_plus_HF"] = df["LF"] + df["HF"]
    df["LF_minus_HF"] = df["LF"] - df["HF"]
    engineered_features.extend(["LF_HF_ratio", "LF_plus_HF", "LF_minus_HF"])

if all(c in df.columns for c in ["Age", "BMI"]):
    df["Age_BMI_interaction"] = df["Age"] * df["BMI"]
    df["Age_squared"] = df["Age"] ** 2
    engineered_features.extend(["Age_BMI_interaction", "Age_squared"])

if "SDNN" in df.columns and "SC" in df.columns:
    df["SDNN_SC_ratio"] = df["SDNN"] / (df["SC"] + 1e-9)
    engineered_features.append("SDNN_SC_ratio")

if "LF" in df.columns and "SDNN" in df.columns:
    df["LF_SDNN_ratio"] = df["LF"] / (df["SDNN"] + 1e-9)
    engineered_features.append("LF_SDNN_ratio")

# 平方根轉換 (處理偏態分佈)
for col in ["HF", "LF", "SDNN"]:
    if col in df.columns:
        df[f"{col}_sqrt"] = np.sqrt(df[col].clip(lower=0))
        engineered_features.append(f"{col}_sqrt")

# 更新特徵列表
ALL_FEATURES_EXT = ALL_FEATURES + [f for f in engineered_features if f in df.columns]
print(f"✓ 新增 {len(engineered_features)} 個工程特徵")
print(f"✓ 總特徵數: {len(ALL_FEATURES_EXT)}")


[步驟 2] 特徵工程
----------------------------------------------------------------------------------------------------
✓ 新增 10 個工程特徵
✓ 總特徵數: 23


In [5]:
# ==================== 3. 缺失值分析與處理 ====================
print("\n[步驟 3] 缺失值分析")
print("-" * 100)

missing_report = pd.DataFrame({
    'feature': [c for c in ALL_FEATURES_EXT if c in df.columns],
    'missing_count': [df[c].isna().sum() for c in ALL_FEATURES_EXT if c in df.columns],
    'missing_pct': [f"{100*df[c].isna().sum()/len(df):.1f}%" for c in ALL_FEATURES_EXT if c in df.columns]
})

print(missing_report.sort_values('missing_count', ascending=False).head(10).to_string(index=False))
missing_report.to_csv(REPORTS_DIR / "missing_values_report.csv", index=False)


[步驟 3] 缺失值分析
----------------------------------------------------------------------------------------------------
      feature  missing_count missing_pct
  LF_minus_HF             63       12.5%
   LF_plus_HF             63       12.5%
  LF_HF_ratio             63       12.5%
LF_SDNN_ratio             55       11.0%
           LF             53       10.6%
      LF_sqrt             53       10.6%
      HF_sqrt             51       10.2%
           HF             51       10.2%
SDNN_SC_ratio             43        8.6%
         LFHF             28        5.6%


In [6]:
# ==================== 4. 診斷分佈分析 ====================
print("\n[步驟 4] 診斷標籤分佈")
print("-" * 100)

label_distribution = []
for label in LABEL_COLS:
    if label in df.columns:
        counts = df[label].value_counts(dropna=False)
        total = df[label].notna().sum()
        n_pos = counts.get(1, 0)
        n_neg = counts.get(0, 0)
        
        label_distribution.append({
            'label': label,
            'total': total,
            'negative': n_neg,
            'positive': n_pos,
            'pos_ratio': f"{100*n_pos/total:.1f}%" if total > 0 else "0%",
            'imbalance_ratio': f"1:{n_neg/n_pos:.1f}" if n_pos > 0 else "N/A"
        })

dist_df = pd.DataFrame(label_distribution)
print(dist_df.to_string(index=False))
dist_df.to_csv(REPORTS_DIR / "label_distribution.csv", index=False)


[步驟 4] 診斷標籤分佈
----------------------------------------------------------------------------------------------------
label  total  negative  positive pos_ratio imbalance_ratio
  SSD    502       378       124     24.7%           1:3.0
  MDD    502       402       100     19.9%           1:4.0
Panic    502       456        46      9.2%           1:9.9
  GAD    502       339       163     32.5%           1:2.1


In [7]:
# ==================== 5. 改進的 PCA 分析 ====================
print("\n[步驟 5] PCA 降維分析")
print("-" * 100)

hrv_cols_available = [c for c in HRV_COLS if c in df.columns]
X_pca = df[hrv_cols_available].copy()

# KNN Imputer
imp = KNNImputer(n_neighbors=5)
X_pca_filled = pd.DataFrame(
    imp.fit_transform(X_pca),
    columns=X_pca.columns,
    index=X_pca.index
)

# RobustScaler
scaler = RobustScaler()
X_pca_scaled = scaler.fit_transform(X_pca_filled)

# PCA
pca = PCA(random_state=42)
X_pca_transformed = pca.fit_transform(X_pca_scaled)

# 解釋變異量
explained_var_df = pd.DataFrame({
    'PC': [f'PC{i+1}' for i in range(min(10, len(pca.explained_variance_ratio_)))],
    'Variance_Ratio': pca.explained_variance_ratio_[:10],
    'Cumulative': np.cumsum(pca.explained_variance_ratio_)[:10]
})

print("\nPCA 解釋變異量 (前10個主成分):")
print(explained_var_df.to_string(index=False))

# 視覺化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
n_show = min(15, len(pca.explained_variance_ratio_))
ax1.bar(range(1, n_show+1), pca.explained_variance_ratio_[:n_show])
ax1.set_xlabel('主成分編號', fontsize=11)
ax1.set_ylabel('解釋變異量比例', fontsize=11)
ax1.set_title('Scree Plot - 各主成分解釋變異量', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

# 累積解釋變異量
ax2.plot(range(1, len(pca.explained_variance_ratio_)+1),
         np.cumsum(pca.explained_variance_ratio_), 'bo-', linewidth=2)
ax2.axhline(y=0.80, color='r', linestyle='--', label='80% 閾值', linewidth=2)
ax2.axhline(y=0.90, color='orange', linestyle='--', label='90% 閾值', linewidth=2)
ax2.set_xlabel('主成分數量', fontsize=11)
ax2.set_ylabel('累積解釋變異量', fontsize=11)
ax2.set_title('累積解釋變異量曲線', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PLOTS_DIR / "pca_variance_analysis.png", dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ PCA 圖表已儲存")

# PCA 散點圖 (按診斷著色)
for label in LABEL_COLS:
    if label not in df.columns:
        continue
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    mask0 = (df[label] == 0)
    mask1 = (df[label] == 1)
    
    # PC1 vs PC2
    axes[0].scatter(X_pca_transformed[mask0, 0], X_pca_transformed[mask0, 1],
                   alpha=0.5, s=40, label=f'{label}=0 (n={mask0.sum()})')
    axes[0].scatter(X_pca_transformed[mask1, 0], X_pca_transformed[mask1, 1],
                   alpha=0.5, s=40, label=f'{label}=1 (n={mask1.sum()})')
    axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})', fontsize=11)
    axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})', fontsize=11)
    axes[0].set_title('PC1 vs PC2', fontsize=12, fontweight='bold')
    axes[0].legend(fontsize=9)
    axes[0].grid(True, alpha=0.3)
    
    # PC1 vs PC3
    if len(pca.explained_variance_ratio_) >= 3:
        axes[1].scatter(X_pca_transformed[mask0, 0], X_pca_transformed[mask0, 2],
                       alpha=0.5, s=40, label=f'{label}=0')
        axes[1].scatter(X_pca_transformed[mask1, 0], X_pca_transformed[mask1, 2],
                       alpha=0.5, s=40, label=f'{label}=1')
        axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})', fontsize=11)
        axes[1].set_ylabel(f'PC3 ({pca.explained_variance_ratio_[2]:.1%})', fontsize=11)
        axes[1].set_title('PC1 vs PC3', fontsize=12, fontweight='bold')
        axes[1].legend(fontsize=9)
        axes[1].grid(True, alpha=0.3)
        
        # PC2 vs PC3
        axes[2].scatter(X_pca_transformed[mask0, 1], X_pca_transformed[mask0, 2],
                       alpha=0.5, s=40, label=f'{label}=0')
        axes[2].scatter(X_pca_transformed[mask1, 1], X_pca_transformed[mask1, 2],
                       alpha=0.5, s=40, label=f'{label}=1')
        axes[2].set_xlabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})', fontsize=11)
        axes[2].set_ylabel(f'PC3 ({pca.explained_variance_ratio_[2]:.1%})', fontsize=11)
        axes[2].set_title('PC2 vs PC3', fontsize=12, fontweight='bold')
        axes[2].legend(fontsize=9)
        axes[2].grid(True, alpha=0.3)
    
    plt.suptitle(f'PCA 分析 - {label}', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / f"pca_scatter_{label}.png", dpi=300, bbox_inches='tight')
    plt.close()

print(f"✓ 已生成 {len(LABEL_COLS)} 個診斷的 PCA 散點圖")


[步驟 5] PCA 降維分析
----------------------------------------------------------------------------------------------------

PCA 解釋變異量 (前10個主成分):
 PC  Variance_Ratio  Cumulative
PC1        0.557316    0.557316
PC2        0.163877    0.721193
PC3        0.125902    0.847095
PC4        0.077201    0.924296
PC5        0.039472    0.963768
PC6        0.022317    0.986085
PC7        0.013915    1.000000
✓ PCA 圖表已儲存
✓ 已生成 4 個診斷的 PCA 散點圖


In [8]:
# ==================== 6. 特徵重要性分析 ====================
print("\n[步驟 6] 特徵重要性分析")
print("-" * 100)

feature_importance_results = {}

for label in LABEL_COLS:
    if label not in df.columns:
        continue
    
    print(f"\n分析 {label} 的特徵重要性...")
    
    mask = df[label].notna()
    available_features = [c for c in ALL_FEATURES_EXT if c in df.columns]
    
    X = df.loc[mask, available_features].copy()
    y = df.loc[mask, label].values
    
    if len(np.unique(y)) < 2 or len(X) < 20:
        print(f"  資料不足，跳過")
        continue
    
    # KNN Imputer
    imp = KNNImputer(n_neighbors=5)
    X_filled = pd.DataFrame(imp.fit_transform(X), columns=X.columns)
    
    # Mutual Information
    mi_scores = mutual_info_classif(X_filled, y, random_state=42)
    
    # Random Forest Importance
    rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_filled, y)
    rf_importance = rf.feature_importances_
    
    # 整合分數
    mi_norm = mi_scores / (mi_scores.max() + 1e-9)
    rf_norm = rf_importance / (rf_importance.max() + 1e-9)
    
    importance_df = pd.DataFrame({
        'feature': X.columns,
        'mutual_info': mi_scores,
        'rf_importance': rf_importance,
        'mi_normalized': mi_norm,
        'rf_normalized': rf_norm,
        'combined_score': (mi_norm + rf_norm) / 2
    }).sort_values('combined_score', ascending=False)
    
    feature_importance_results[label] = importance_df
    
    print(f"  Top 10 重要特徵:")
    print(importance_df.head(10)[['feature', 'combined_score']].to_string(index=False))
    
    # 儲存
    importance_df.to_csv(REPORTS_DIR / f"feature_importance_{label}.csv", index=False)
    
    # 視覺化
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    top_n = min(15, len(importance_df))
    top_features = importance_df.head(top_n)
    
    # Mutual Information
    axes[0].barh(range(top_n), top_features['mutual_info'].values, color='steelblue')
    axes[0].set_yticks(range(top_n))
    axes[0].set_yticklabels(top_features['feature'].values, fontsize=9)
    axes[0].set_xlabel('Mutual Information Score', fontsize=11)
    axes[0].set_title(f'{label} - Mutual Information', fontsize=12, fontweight='bold')
    axes[0].invert_yaxis()
    axes[0].grid(True, alpha=0.3, axis='x')
    
    # Random Forest
    axes[1].barh(range(top_n), top_features['rf_importance'].values, color='coral')
    axes[1].set_yticks(range(top_n))
    axes[1].set_yticklabels(top_features['feature'].values, fontsize=9)
    axes[1].set_xlabel('Random Forest Importance', fontsize=11)
    axes[1].set_title(f'{label} - Random Forest', fontsize=12, fontweight='bold')
    axes[1].invert_yaxis()
    axes[1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / f"feature_importance_{label}.png", dpi=300, bbox_inches='tight')
    plt.close()

print(f"\n✓ 特徵重要性分析完成")


[步驟 6] 特徵重要性分析
----------------------------------------------------------------------------------------------------

分析 SSD 的特徵重要性...
  Top 10 重要特徵:
            feature  combined_score
                BMI        0.777506
               LFHF        0.734781
Age_BMI_interaction        0.690302
                 SC        0.601685
                RSA        0.563007
               SDNN        0.374005
                Age        0.360884
                 FT        0.345366
        LF_HF_ratio        0.323478
      SDNN_SC_ratio        0.285809

分析 MDD 的特徵重要性...
  Top 10 重要特徵:
            feature  combined_score
                Age        0.884025
            HF_sqrt        0.820592
                 HF        0.739043
                 SC        0.734665
        Age_squared        0.602606
                 FT        0.573825
                BMI        0.566743
      SDNN_SC_ratio        0.555450
         LF_plus_HF        0.518636
Age_BMI_interaction        0.501345

分析 Panic 的特徵重要性...
  Top

In [9]:
# ==================== 7. 進階機器學習模型 ====================
print("\n[步驟 7] 訓練與評估機器學習模型")
print("-" * 100)

def optimize_threshold(y_true, y_pred_proba, metric='f1'):
    """尋找最佳決策閾值"""
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred_proba)
    
    if metric == 'f1':
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
        best_idx = np.argmax(f1_scores[:-1])
    elif metric == 'balanced':
        # 平衡精確率和召回率
        balanced_scores = np.sqrt(precision * recall)
        best_idx = np.argmax(balanced_scores[:-1])
    
    return thresholds[best_idx] if len(thresholds) > 0 else 0.5

def comprehensive_evaluation(model, X_test, y_test, model_name, label_name, save_dir):
    """全面評估模型"""
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 找最佳閾值
    best_threshold = optimize_threshold(y_test, y_pred_proba, metric='f1')
    
    # 預測
    y_pred_05 = (y_pred_proba >= 0.5).astype(int)
    y_pred_opt = (y_pred_proba >= best_threshold).astype(int)
    
    # 計算指標
    metrics = {
        'auc': roc_auc_score(y_test, y_pred_proba),
        'ap': average_precision_score(y_test, y_pred_proba),
        'f1_threshold_05': f1_score(y_test, y_pred_05),
        'f1_threshold_opt': f1_score(y_test, y_pred_opt),
        'best_threshold': best_threshold
    }
    
    # 視覺化
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # ROC Curve
    ax1 = fig.add_subplot(gs[0, 0])
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    ax1.plot(fpr, tpr, linewidth=2, label=f'AUC = {metrics["auc"]:.3f}')
    ax1.plot([0, 1], [0, 1], 'k--', linewidth=1)
    ax1.set_xlabel('False Positive Rate', fontsize=10)
    ax1.set_ylabel('True Positive Rate', fontsize=10)
    ax1.set_title('ROC Curve', fontsize=11, fontweight='bold')
    ax1.legend(fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # PR Curve
    ax2 = fig.add_subplot(gs[0, 1])
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    ax2.plot(recall, precision, linewidth=2, label=f'AP = {metrics["ap"]:.3f}')
    ax2.set_xlabel('Recall', fontsize=10)
    ax2.set_ylabel('Precision', fontsize=10)
    ax2.set_title('Precision-Recall Curve', fontsize=11, fontweight='bold')
    ax2.legend(fontsize=9)
    ax2.grid(True, alpha=0.3)
    
    # Confusion Matrix (threshold = 0.5)
    ax3 = fig.add_subplot(gs[0, 2])
    cm_05 = confusion_matrix(y_test, y_pred_05)
    sns.heatmap(cm_05, annot=True, fmt='d', cmap='Blues', ax=ax3, cbar=False)
    ax3.set_xlabel('Predicted', fontsize=10)
    ax3.set_ylabel('Actual', fontsize=10)
    ax3.set_title('Confusion Matrix (threshold=0.5)', fontsize=11, fontweight='bold')
    
    # Confusion Matrix (optimal threshold)
    ax4 = fig.add_subplot(gs[1, 0])
    cm_opt = confusion_matrix(y_test, y_pred_opt)
    sns.heatmap(cm_opt, annot=True, fmt='d', cmap='Greens', ax=ax4, cbar=False)
    ax4.set_xlabel('Predicted', fontsize=10)
    ax4.set_ylabel('Actual', fontsize=10)
    ax4.set_title(f'Confusion Matrix (threshold={best_threshold:.3f})', fontsize=11, fontweight='bold')
    
    # Prediction Distribution
    ax5 = fig.add_subplot(gs[1, 1])
    ax5.hist(y_pred_proba[y_test == 0], bins=30, alpha=0.6, label='Negative', density=True, color='blue')
    ax5.hist(y_pred_proba[y_test == 1], bins=30, alpha=0.6, label='Positive', density=True, color='red')
    ax5.axvline(x=0.5, color='black', linestyle='--', linewidth=1, label='Default (0.5)')
    ax5.axvline(x=best_threshold, color='green', linestyle='--', linewidth=2, label=f'Optimal ({best_threshold:.3f})')
    ax5.set_xlabel('Predicted Probability', fontsize=10)
    ax5.set_ylabel('Density', fontsize=10)
    ax5.set_title('Prediction Distribution', fontsize=11, fontweight='bold')
    ax5.legend(fontsize=8)
    ax5.grid(True, alpha=0.3)
    
    # Threshold vs F1
    ax6 = fig.add_subplot(gs[1, 2])
    thresholds_range = np.linspace(0.1, 0.9, 50)
    f1_scores = []
    for t in thresholds_range:
        y_pred_t = (y_pred_proba >= t).astype(int)
        f1_scores.append(f1_score(y_test, y_pred_t))
    
    ax6.plot(thresholds_range, f1_scores, linewidth=2)
    ax6.axvline(x=best_threshold, color='r', linestyle='--', linewidth=2, label=f'Best = {best_threshold:.3f}')
    ax6.set_xlabel('Threshold', fontsize=10)
    ax6.set_ylabel('F1 Score', fontsize=10)
    ax6.set_title('Threshold vs F1 Score', fontsize=11, fontweight='bold')
    ax6.legend(fontsize=9)
    ax6.grid(True, alpha=0.3)
    
    # Performance Metrics Text
    ax7 = fig.add_subplot(gs[2, :])
    ax7.axis('off')
    
    metrics_text = f"""
    模型: {model_name} | 診斷標籤: {label_name} | 測試集大小: {len(y_test)}
    
    ROC-AUC: {metrics['auc']:.4f}
    Average Precision: {metrics['ap']:.4f}
    
    F1 Score (threshold=0.5): {metrics['f1_threshold_05']:.4f}
    F1 Score (threshold={best_threshold:.3f}): {metrics['f1_threshold_opt']:.4f}
    
    最佳閾值: {best_threshold:.4f}
    
    分類報告 (使用最佳閾值):
    {classification_report(y_test, y_pred_opt, target_names=['Negative', 'Positive'], digits=3)}
    """
    
    ax7.text(0.1, 0.5, metrics_text, fontsize=9, family='monospace',
             verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.suptitle(f'{model_name} - {label_name} 完整評估報告', 
                 fontsize=14, fontweight='bold', y=0.98)
    
    plt.savefig(save_dir / f"{label_name}_{model_name}_evaluation.png",
                dpi=300, bbox_inches='tight')
    plt.close()
    
    return metrics

def train_improved_models(df, label, feature_cols, output_dir):
    """
    訓練改進的模型 - 防過擬合版本
    """
    print(f"\n{'='*80}")
    print(f"處理診斷標籤: {label}")
    print(f"{'='*80}")
    
    # 準備資料
    mask = df[label].notna()
    X = df.loc[mask, feature_cols].copy()
    y = df.loc[mask, label].values.astype(int)
    
    if len(np.unique(y)) < 2 or len(X) < 50:
        print(f"⚠ 資料不足 (樣本數={len(X)})，跳過")
        return None
    
    print(f"✓ 總樣本數: {len(X)}")
    print(f"✓ 正類比例: {np.mean(y):.2%} ({np.sum(y)}/{len(y)})")
    
    # KNN Imputer
    imp = KNNImputer(n_neighbors=5)
    X_filled = pd.DataFrame(imp.fit_transform(X), columns=X.columns)
    
    # 特徵選擇 (使用前期分析的重要特徵)
    if label in feature_importance_results:
        top_features = feature_importance_results[label].head(12)['feature'].tolist()
        selected_cols = [c for c in top_features if c in X_filled.columns]
        
        if len(selected_cols) >= 5:
            X_filled = X_filled[selected_cols]
            print(f"✓ 使用前 {len(selected_cols)} 個重要特徵")
    
    # 分割資料
    X_train, X_test, y_train, y_test = train_test_split(
        X_filled, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"✓ 訓練集: {len(X_train)}, 測試集: {len(X_test)}")
    
    # 標準化 (使用 RobustScaler)
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # SMOTE (如果需要)
    imbalance_ratio = np.sum(y_train == 0) / max(np.sum(y_train == 1), 1)
    
    if imbalance_ratio > 2 and np.sum(y_train == 1) >= 6:
        print(f"✓ 不平衡比例: 1:{imbalance_ratio:.1f}，使用 SMOTE")
        
        try:
            smote = SMOTE(random_state=42, k_neighbors=min(5, np.sum(y_train == 1) - 1))
            X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
            print(f"✓ SMOTE 後樣本數: {len(y_train)} → {len(y_train_balanced)}")
        except Exception as e:
            print(f"⚠ SMOTE 失敗: {e}，使用原始資料")
            X_train_balanced, y_train_balanced = X_train_scaled, y_train
    else:
        X_train_balanced, y_train_balanced = X_train_scaled, y_train
    
    # 定義模型 (防過擬合配置)
    models = {
        'Logistic_L1': LogisticRegression(
            penalty='l1',
            C=0.1,
            solver='saga',
            max_iter=3000,
            class_weight='balanced',
            random_state=42
        ),
        'Logistic_L2': LogisticRegression(
            penalty='l2',
            C=1.0,
            solver='lbfgs',
            max_iter=3000,
            class_weight='balanced',
            random_state=42
        ),
        'RandomForest_Simple': RandomForestClassifier(
            n_estimators=100,
            max_depth=5,
            min_samples_split=20,
            min_samples_leaf=10,
            max_features='sqrt',
            class_weight='balanced',
            random_state=42,
            n_jobs=-1
        ),
        'GradientBoosting_Simple': GradientBoostingClassifier(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.05,
            min_samples_split=20,
            min_samples_leaf=10,
            subsample=0.8,
            random_state=42
        ),
        'SVM_RBF': SVC(
            kernel='rbf',
            C=1.0,
            gamma='scale',
            probability=True,
            class_weight='balanced',
            random_state=42
        )
    }
    
    # 訓練和評估所有模型
    results = {}
    
    for model_name, model in models.items():
        print(f"\n--- {model_name} ---")
        
        # 交叉驗證
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_validate(
            model, X_train_balanced, y_train_balanced,
            cv=cv,
            scoring={
                'roc_auc': 'roc_auc',
                'average_precision': 'average_precision',
                'f1': 'f1'
            },
            n_jobs=-1,
            return_train_score=True
        )
        
        # 訓練最終模型
        model.fit(X_train_balanced, y_train_balanced)
        
        # 測試集評估
        test_metrics = comprehensive_evaluation(
            model, X_test_scaled, y_test,
            model_name, label, output_dir
        )
        
        # 整合結果
        results[model_name] = {
            'cv_auc_mean': np.mean(cv_scores['test_roc_auc']),
            'cv_auc_std': np.std(cv_scores['test_roc_auc']),
            'cv_ap_mean': np.mean(cv_scores['test_average_precision']),
            'cv_f1_mean': np.mean(cv_scores['test_f1']),
            'train_auc_mean': np.mean(cv_scores['train_roc_auc']),
            'test_auc': test_metrics['auc'],
            'test_ap': test_metrics['ap'],
            'test_f1_opt': test_metrics['f1_threshold_opt'],
            'best_threshold': test_metrics['best_threshold'],
            'overfitting_gap': np.mean(cv_scores['train_roc_auc']) - test_metrics['auc']
        }
        
        print(f"  CV AUC: {results[model_name]['cv_auc_mean']:.3f} ± {results[model_name]['cv_auc_std']:.3f}")
        print(f"  Test AUC: {test_metrics['auc']:.3f}")
        print(f"  Test AP: {test_metrics['ap']:.3f}")
        print(f"  Test F1 (優化閾值): {test_metrics['f1_threshold_opt']:.3f}")
        print(f"  過擬合程度: {results[model_name]['overfitting_gap']:.3f}")
    
    # 找最佳模型
    best_model_name = max(results.items(), key=lambda x: x[1]['test_auc'])[0]
    best_model = models[best_model_name]
    
    print(f"\n{'='*80}")
    print(f"✓ 最佳模型: {best_model_name}")
    print(f"  Test AUC: {results[best_model_name]['test_auc']:.3f}")
    print(f"  Test AP: {results[best_model_name]['test_ap']:.3f}")
    print(f"  Test F1: {results[best_model_name]['test_f1_opt']:.3f}")
    print(f"{'='*80}")
    
    # 儲存最佳模型
    model_save_path = MODELS_DIR / f"{label}_best_model.pkl"
    joblib.dump({
        'model': best_model,
        'scaler': scaler,
        'imputer': imp,
        'feature_names': X_filled.columns.tolist(),
        'threshold': results[best_model_name]['best_threshold']
    }, model_save_path)
    
    print(f"✓ 模型已儲存: {model_save_path}")
    
    return results

# 對每個診斷執行分析
all_results = {}

for label in LABEL_COLS:
    if label not in df.columns:
        continue
    
    available_features = [c for c in ALL_FEATURES_EXT if c in df.columns]
    
    results = train_improved_models(df, label, available_features, PLOTS_DIR)
    
    if results:
        all_results[label] = results


[步驟 7] 訓練與評估機器學習模型
----------------------------------------------------------------------------------------------------

處理診斷標籤: SSD
✓ 總樣本數: 502
✓ 正類比例: 24.70% (124/502)
✓ 使用前 12 個重要特徵
✓ 訓練集: 401, 測試集: 101
✓ 不平衡比例: 1:3.1，使用 SMOTE
✓ SMOTE 後樣本數: 401 → 604

--- Logistic_L1 ---
  CV AUC: 0.633 ± 0.044
  Test AUC: 0.610
  Test AP: 0.333
  Test F1 (優化閾值): 0.459
  過擬合程度: 0.041

--- Logistic_L2 ---
  CV AUC: 0.645 ± 0.024
  Test AUC: 0.636
  Test AP: 0.386
  Test F1 (優化閾值): 0.483
  過擬合程度: 0.035

--- RandomForest_Simple ---
  CV AUC: 0.784 ± 0.020
  Test AUC: 0.646
  Test AP: 0.436
  Test F1 (優化閾值): 0.491
  過擬合程度: 0.281

--- GradientBoosting_Simple ---
  CV AUC: 0.778 ± 0.027
  Test AUC: 0.629
  Test AP: 0.391
  Test F1 (優化閾值): 0.468
  過擬合程度: 0.342

--- SVM_RBF ---
  CV AUC: 0.756 ± 0.039
  Test AUC: 0.618
  Test AP: 0.349
  Test F1 (優化閾值): 0.457
  過擬合程度: 0.236

✓ 最佳模型: RandomForest_Simple
  Test AUC: 0.646
  Test AP: 0.436
  Test F1: 0.491
✓ 模型已儲存: D:\FLY114\Analysis_Results_20251003_135729\t

In [10]:
# ==================== 8. 結果彙整 ====================
print("\n[步驟 8] 結果彙整與報告生成")
print("-" * 100)

# 建立比較表格
comparison_rows = []
for label, model_results in all_results.items():
    for model_name, metrics in model_results.items():
        comparison_rows.append({
            'Label': label,
            'Model': model_name,
            'CV_AUC_Mean': f"{metrics['cv_auc_mean']:.3f}",
            'CV_AUC_Std': f"{metrics['cv_auc_std']:.3f}",
            'Test_AUC': f"{metrics['test_auc']:.3f}",
            'Test_AP': f"{metrics['test_ap']:.3f}",
            'Test_F1': f"{metrics['test_f1_opt']:.3f}",
            'Best_Threshold': f"{metrics['best_threshold']:.3f}",
            'Overfitting_Gap': f"{metrics['overfitting_gap']:.3f}"
        })

comparison_df = pd.DataFrame(comparison_rows)

print("\n完整模型比較結果:")
print(comparison_df.to_string(index=False))

# 儲存比較結果
comparison_df.to_csv(REPORTS_DIR / "model_comparison_full.csv", index=False)
comparison_df.to_excel(OUT_DIR / "model_comparison.xlsx", index=False, engine='openpyxl')

# 生成最佳模型摘要
best_models_summary = []
for label in all_results.keys():
    results = all_results[label]
    best_model = max(results.items(), key=lambda x: x[1]['test_auc'])
    
    best_models_summary.append({
        'Label': label,
        'Best_Model': best_model[0],
        'Test_AUC': f"{best_model[1]['test_auc']:.3f}",
        'Test_AP': f"{best_model[1]['test_ap']:.3f}",
        'Test_F1': f"{best_model[1]['test_f1_opt']:.3f}",
        'CV_AUC': f"{best_model[1]['cv_auc_mean']:.3f} ± {best_model[1]['cv_auc_std']:.3f}",
        'Overfitting': f"{best_model[1]['overfitting_gap']:.3f}"
    })

best_summary_df = pd.DataFrame(best_models_summary)

print("\n最佳模型摘要:")
print(best_summary_df.to_string(index=False))

best_summary_df.to_csv(REPORTS_DIR / "best_models_summary.csv", index=False)

# 視覺化最佳模型比較
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

labels_list = list(all_results.keys())
metrics_to_plot = ['test_auc', 'test_ap', 'test_f1_opt', 'overfitting_gap']
titles = ['Test AUC', 'Test AP (Average Precision)', 'Test F1 (Optimized)', 'Overfitting Gap']
colors = ['steelblue', 'coral', 'mediumseagreen', 'indianred']

for idx, (metric, title, color) in enumerate(zip(metrics_to_plot, titles, colors)):
    ax = axes[idx // 2, idx % 2]
    
    # 取得每個標籤的所有模型該指標
    for label_idx, label in enumerate(labels_list):
        results = all_results[label]
        model_names = list(results.keys())
        metric_values = [results[m][metric] for m in model_names]
        
        x_positions = np.arange(len(model_names)) + label_idx * 0.15
        ax.bar(x_positions, metric_values, width=0.15, 
               label=label, alpha=0.8)
    
    ax.set_xlabel('Models', fontsize=11)
    ax.set_ylabel(title, fontsize=11)
    ax.set_title(f'{title} - 所有診斷比較', fontsize=12, fontweight='bold')
    ax.set_xticks(np.arange(len(model_names)) + 0.225)
    ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=9)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')
    
    if metric != 'overfitting_gap':
        ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig(PLOTS_DIR / "all_models_comparison.png", dpi=300, bbox_inches='tight')
plt.close()

print(f"\n✓ 比較圖表已儲存")


[步驟 8] 結果彙整與報告生成
----------------------------------------------------------------------------------------------------

完整模型比較結果:
Label                   Model CV_AUC_Mean CV_AUC_Std Test_AUC Test_AP Test_F1 Best_Threshold Overfitting_Gap
  SSD             Logistic_L1       0.633      0.044    0.610   0.333   0.459          0.541           0.041
  SSD             Logistic_L2       0.645      0.024    0.636   0.386   0.483          0.587           0.035
  SSD     RandomForest_Simple       0.784      0.020    0.646   0.436   0.491          0.578           0.281
  SSD GradientBoosting_Simple       0.778      0.027    0.629   0.391   0.468          0.477           0.342
  SSD                 SVM_RBF       0.756      0.039    0.618   0.349   0.457          0.253           0.236
  MDD             Logistic_L1       0.704      0.033    0.540   0.227   0.377          0.565           0.183
  MDD             Logistic_L2       0.718      0.033    0.569   0.240   0.382          0.483           0.16

In [11]:
# ==================== 9. 生成最終報告 ====================
print("\n[步驟 9] 生成最終分析報告")
print("-" * 100)

report_content = f"""
{'='*100}
機器學習分析完整報告
{'='*100}

執行時間: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
資料來源: {XLSX_PATH}

{'='*100}
1. 資料概況
{'='*100}

原始樣本數: {len(df)}
特徵總數: {len(ALL_FEATURES_EXT)}
  - 基本特徵: {len(BASIC_COLS)}
  - 控制變數: {len(CONTROL_COLS)}
  - HRV 特徵: {len(HRV_COLS)}
  - 工程特徵: {len(engineered_features)}

診斷標籤分佈:
{dist_df.to_string(index=False)}

{'='*100}
2. 最佳模型表現
{'='*100}

{best_summary_df.to_string(index=False)}

{'='*100}
3. 關鍵發現
{'='*100}

"""

# 添加每個診斷的關鍵發現
for label in all_results.keys():
    results = all_results[label]
    best_model = max(results.items(), key=lambda x: x[1]['test_auc'])
    
    report_content += f"""
{label}:
  ✓ 最佳模型: {best_model[0]}
  ✓ Test AUC: {best_model[1]['test_auc']:.3f}
  ✓ Test AP: {best_model[1]['test_ap']:.3f}
  ✓ Test F1: {best_model[1]['test_f1_opt']:.3f}
  ✓ 最佳閾值: {best_model[1]['best_threshold']:.3f}
  ✓ 過擬合程度: {best_model[1]['overfitting_gap']:.3f}
  
  前5重要特徵:
"""
    
    if label in feature_importance_results:
        top_5 = feature_importance_results[label].head(5)
        for i, row in top_5.iterrows():
            report_content += f"    {row['feature']}: {row['combined_score']:.3f}\n"
    
    report_content += "\n"

report_content += f"""
{'='*100}
4. 改善建議
{'='*100}

基於分析結果，以下是針對每個診斷的具體建議:

"""

for label in all_results.keys():
    results = all_results[label]
    best_model = max(results.items(), key=lambda x: x[1]['test_auc'])
    
    auc = best_model[1]['test_auc']
    gap = best_model[1]['overfitting_gap']
    
    report_content += f"{label}:\n"
    
    if auc >= 0.7:
        report_content += f"  ✓ 表現良好 (AUC={auc:.3f})\n"
    elif auc >= 0.6:
        report_content += f"  ⚠ 表現中等 (AUC={auc:.3f})，建議收集更多資料或新增特徵\n"
    else:
        report_content += f"  ✗ 表現較差 (AUC={auc:.3f})，需要重新評估特徵或資料品質\n"
    
    if gap > 0.15:
        report_content += f"  ⚠ 有過擬合傾向 (差距={gap:.3f})，建議增加正則化或減少特徵\n"
    else:
        report_content += f"  ✓ 過擬合控制良好 (差距={gap:.3f})\n"
    
    report_content += "\n"

report_content += f"""
{'='*100}
5. 輸出檔案
{'='*100}

報告位置: {REPORTS_DIR}
圖表位置: {PLOTS_DIR}
模型位置: {MODELS_DIR}

主要輸出:
  - model_comparison.xlsx: 完整模型比較
  - best_models_summary.csv: 最佳模型摘要
  - feature_importance_*.csv: 各診斷的特徵重要性
  - *_evaluation.png: 詳細評估圖表
  - *_best_model.pkl: 訓練好的模型

{'='*100}
分析完成!
{'='*100}
"""

# 儲存報告
report_path = REPORTS_DIR / "analysis_report.txt"
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report_content)

print(report_content)
print(f"\n✓ 完整報告已儲存: {report_path}")

# ==================== 10. 如何使用訓練好的模型進行預測 ====================
print("\n[步驟 10] 模型使用說明")
print("-" * 100)

usage_guide = """
如何使用訓練好的模型進行預測:

```python
import joblib
import pandas as pd
import numpy as np

# 1. 載入模型
model_data = joblib.load('trained_models/GAD_best_model.pkl')
model = model_data['model']
scaler = model_data['scaler']
imputer = model_data['imputer']
feature_names = model_data['feature_names']
threshold = model_data['threshold']

# 2. 準備新資料
new_data = pd.DataFrame({
    'Age': [45],
    'Sex': [1],
    'BMI': [24.5],
    # ... 其他特徵
})

# 3. 確保特徵順序一致
new_data = new_data[feature_names]

# 4. 填補缺失值
new_data_imputed = imputer.transform(new_data)

# 5. 標準化
new_data_scaled = scaler.transform(new_data_imputed)

# 6. 預測
pred_proba = model.predict_proba(new_data_scaled)[:, 1]
pred_class = (pred_proba >= threshold).astype(int)

print(f"預測機率: {pred_proba[0]:.3f}")
print(f"預測類別: {'陽性' if pred_class[0] == 1 else '陰性'}")
```
"""

print(usage_guide)

usage_path = REPORTS_DIR / "model_usage_guide.txt"
with open(usage_path, 'w', encoding='utf-8') as f:
    f.write(usage_guide)

print(f"\n✓ 使用說明已儲存: {usage_path}")

# ==================== 完成 ====================
print("\n" + "="*100)
print("🎉 分析流程完成!")
print("="*100)
print(f"\n所有結果已儲存至: {OUT_DIR}")
print(f"  📊 圖表: {PLOTS_DIR}")
print(f"  🤖 模型: {MODELS_DIR}")
print(f"  📄 報告: {REPORTS_DIR}")
print("\n感謝使用!")
print("="*100)


[步驟 9] 生成最終分析報告
----------------------------------------------------------------------------------------------------

機器學習分析完整報告

執行時間: 2025-10-03 13:58:02
資料來源: D:\FLY114\Diagnosis and autonomic marker data for VNS research_20250813.xlsx

1. 資料概況

原始樣本數: 502
特徵總數: 23
  - 基本特徵: 3
  - 控制變數: 3
  - HRV 特徵: 7
  - 工程特徵: 10

診斷標籤分佈:
label  total  negative  positive pos_ratio imbalance_ratio
  SSD    502       378       124     24.7%           1:3.0
  MDD    502       402       100     19.9%           1:4.0
Panic    502       456        46      9.2%           1:9.9
  GAD    502       339       163     32.5%           1:2.1

2. 最佳模型表現

Label              Best_Model Test_AUC Test_AP Test_F1        CV_AUC Overfitting
  SSD     RandomForest_Simple    0.646   0.436   0.491 0.784 ± 0.020       0.281
  MDD                 SVM_RBF    0.601   0.259   0.392 0.839 ± 0.033       0.290
Panic     RandomForest_Simple    0.603   0.148   0.267 0.881 ± 0.008       0.346
  GAD GradientBoosting_Simple    0.623 